In [11]:
'''
Import necessary packages
'''
import os
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import csv
import matplotlib.pyplot as plt
import imageio
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from PIL import Image
# import cv2
from collections import Counter

np.set_printoptions(precision=3)

%matplotlib inline

In [2]:
def convert_to_lab(arr_rgb):
    '''
    This function will transform every rgb value in the list to lab. 
    '''
    
    list_lab = []
    for row in arr_rgb:
        r, g, b = row
        list_lab.append(convert_color(sRGBColor(r, g, b, is_upscaled=True), LabColor).get_value_tuple())
    return np.array(list_lab)


def convert_to_rgb(arr_lab):
    '''
    This function will transform every rgb value in the list to lab. 
    '''
    
    list_rgb = []
    for row in arr_lab:
        l, a, b = row
        list_rgb.append(convert_color(LabColor(l, a, b), sRGBColor).get_value_tuple())
    return np.array(list_rgb)

def find_the_cloeset_iscc(arr_img_lab, arr_iscc_lab):
    
    return arr_iscc_lab[np.argmin(cdist(arr_img_lab, arr_iscc_lab), 1)] 

# function to return key for any value 
def get_key(my_dict, val): 
    for key, value in my_dict.items(): 
         if val == value: 
                return key 
    return "key doesn't exist"
      

In [3]:
'''
Define the color card with ISCC_NBS
'''

# Read ISCC from local file
path_iscc = os.path.join('ISCC_NBS', 'ISCC_NBS_w3.csv')
df_iscc = pd.read_csv(path_iscc, names = ['color_name', 'hex'])

# Get the RGB and CIE-Lab values for each color in ISCC color card
df_iscc['RGB'] = ""
df_iscc['Lab'] = ""
for index, row in df_iscc.iterrows(): 
    df_iscc.loc[index, 'RGB'] = sRGBColor.new_from_rgb_hex(row['hex'])
    df_iscc.loc[index, 'Lab'] = convert_color(df_iscc.loc[index, 'RGB'], LabColor).get_value_tuple()

In [4]:
'''
Define the color card with ISCC_NBS for only red colors
'''

# Read ISCC from local file
path_red_iscc = os.path.join('ISCC_NBS', 'ISCC_NBS_ONLY_RED_w3.csv')
df_red_iscc = pd.read_csv(path_red_iscc, names = ['color_name', 'hex'])

# Get the RGB and CIE-Lab values for each color in ISCC color card
df_red_iscc['RGB'] = ""
df_red_iscc['Lab'] = ""
for index, row in df_red_iscc.iterrows(): 
    df_red_iscc.loc[index, 'RGB'] = sRGBColor.new_from_rgb_hex(row['hex'])
    df_red_iscc.loc[index, 'Lab'] = convert_color(df_iscc.loc[index, 'RGB'], LabColor).get_value_tuple()

In [5]:
def img_to_vec(img_path):
    
    img = Image.open(path_img).convert('RGB')
    img = np.array(img)
    arr_img_lab = convert_to_lab(img.reshape(img.shape[0]*img.shape[1], 3))
    arr_iscc_lab = np.array(df_iscc['Lab'].tolist())
    quantized_img_lab = find_the_cloeset_iscc(arr_img_lab, arr_iscc_lab)
    quantized_img_rgb = convert_to_rgb(quantized_img_lab).reshape(img.shape[0], img.shape[1], 3)
    background_lab = np.array([95.7931405 , -0.15465805, -0.6054342])
    (unique, counts) = np.unique(quantized_img_lab, return_counts=True, axis=0)
    indices = np.argsort(counts)[::-1]
    dict_color_count = {}
    dict_color_name = {}
    sum_counts = sum(counts)


    for i in indices:
        if cdist(unique[i].reshape(1, 3), background_lab.reshape(1, 3)) < 0.001:
            continue
        index_iscc = np.where(cdist(df_iscc['Lab'].tolist(), unique[i].reshape(1, 3))<0.01)
        hex_value = df_iscc.loc[index_iscc[0][0], 'hex']
        color_name = df_iscc.loc[index_iscc[0][0], 'color_name']
        dict_color_count[hex_value] = counts[i]/sum_counts
        dict_color_name[hex_value] = color_name
    return dict_color_count, dict_color_name

In [6]:
def img_to_red_vec(img_path):
    
    img = Image.open(path_img).convert('RGB')
    img = np.array(img)
    arr_img_lab = convert_to_lab(img.reshape(img.shape[0]*img.shape[1], 3))
    arr_iscc_lab = np.array(df_red_iscc['Lab'].tolist())
    quantized_img_lab = find_the_cloeset_iscc(arr_img_lab, arr_iscc_lab)
    quantized_img_rgb = convert_to_rgb(quantized_img_lab).reshape(img.shape[0], img.shape[1], 3)
    background_lab = np.array([95.7931405 , -0.15465805, -0.6054342])
    (unique, counts) = np.unique(quantized_img_lab, return_counts=True, axis=0)
    indices = np.argsort(counts)[::-1]
    dict_color_count = {}
    dict_color_name = {}
    sum_counts = sum(counts)


    for i in indices:
        if cdist(unique[i].reshape(1, 3), background_lab.reshape(1, 3)) < 0.001:
            continue
        index_iscc = np.where(cdist(df_red_iscc['Lab'].tolist(), unique[i].reshape(1, 3))<0.01)
        hex_value = df_red_iscc.loc[index_iscc[0][0], 'hex']
        color_name = df_red_iscc.loc[index_iscc[0][0], 'color_name']
        dict_color_count[hex_value] = counts[i]/sum_counts
        dict_color_name[hex_value] = color_name
    return dict_color_count, dict_color_name

In [6]:

'''
Put all color values into csv for blood images
'''

path = os.path.join('images', 'blood')
df_vec = pd.DataFrame(columns = df_iscc['hex'].values) 
for img_name in os.listdir(path):
    if img_name.startswith('data'):
        continue
        
    path_img = os.path.join(path, img_name)
    dict_color, dict_temp_names = img_to_vec(path_img)
    print(dict_temp_names)
    for color_hex in df_vec.columns:
        if color_hex not in dict_color.keys():
            dict_color[color_hex] = 0
    df_vec = df_vec.append(dict_color,ignore_index=True,sort=False)
    print(df_vec)
    print(df_vec.size)

9;: &#39;Light_Grayish_Brown&#39;, &#39;#57554C&#39;: &#39;Olive_Gray&#39;, &#39;#3E322C&#39;: &#39;Dark_Grayish_Brown&#39;, &#39;#555555&#39;: &#39;Dark_Gray&#39;, &#39;#8DA399&#39;: &#39;Pale_Green&#39;, &#39;#635147&#39;: &#39;Grayish_Brown&#39;, &#39;#3B3121&#39;: &#39;Dark_Olive_Brown&#39;, &#39;#AE9B82&#39;: &#39;Light_Grayish_Yellowish_Brown&#39;, &#39;#483C32&#39;: &#39;Dark_Grayish_Yellowish_Brown&#39;, &#39;#BFB8A5&#39;: &#39;Yellowish_Gray&#39;, &#39;#5B504F&#39;: &#39;Brownish_Gray&#39;, &#39;#E8E3E5&#39;: &#39;Purplish_White&#39;, &#39;#8C8767&#39;: &#39;Light_Grayish_Olive&#39;, &#39;#25241D&#39;: &#39;Olive_Black&#39;, &#39;#BFB9BD&#39;: &#39;Light_Purplish_Gray&#39;, &#39;#593319&#39;: &#39;Deep_Brown&#39;, &#39;#C1B6B3&#39;: &#39;Pinkish_Gray&#39;, &#39;#C2AC99&#39;: &#39;Brownish_Pink&#39;, &#39;#E9E9ED&#39;: &#39;Bluish_White&#39;, &#39;#403D21&#39;: &#39;Dark_Olive&#39;, &#39;#B4BCC0&#39;: &#39;Light_Bluish_Gray&#39;, &#39;#3E1D1E&#39;: &#39;Dark_Reddish_Brown&#39;,

KeyboardInterrupt: 

In [8]:
'''
Put all red color values into csv for blood images
'''
path = os.path.join('images', 'blood')
df_red_vec = pd.DataFrame(columns = df_red_iscc['hex'].values) 
for img_name in os.listdir(path):
    if img_name.startswith('data'):
        continue
        
    path_img = os.path.join(path, img_name)
    print(path_img)
    dict_red, dict_temp_names = img_to_red_vec(path_img)
    for color_hex in df_red_vec.columns:
        if color_hex not in dict_red.keys():
            dict_red[color_hex] = 0
    df_red_vec = df_red_vec.append(dict_red,ignore_index=True,sort=False)
    print(df_red_vec)
    print(df_red_vec.size)

images/blood/online_v1_00190.png
{1: 'online_v1_00190.png'}
   #BE0032  #BC3F4A  #841B2D  #5C0923  #AB4E52  #722F37  #3F1728  #AD8884  \
0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

   #905D5D  #543D3F  ...   #836479   #CE4676   #B3446C   #78184A   #54133B  \
0      0.0      0.0  ...  0.261377  0.125202  0.061941  0.007914  0.031781   

    #A8516E   #673147   #38152C   #AF868E   #915F6D  
0  0.141491  0.020435  0.008028  0.000393  0.002226  

[1 rows x 47 columns]
47
images/blood/online_v1_00224.png
{1: 'online_v1_00190.png', 2: 'online_v1_00224.png'}
   #BE0032  #BC3F4A  #841B2D  #5C0923  #AB4E52  #722F37   #3F1728   #AD8884  \
0      0.0      0.0      0.0      0.0      0.0      0.0  0.000000  0.000000   
1      0.0      0.0      0.0      0.0      0.0      0.0  0.011853  0.000047   

    #905D5D   #543D3F  ...   #836479   #CE4676   #B3446C   #78184A   #54133B  \
0  0.000000  0.000000  ...  0.261377  0.125202  0.061941  0.007914  0.031781   
1  0.01569

KeyboardInterrupt: 

In [13]:
'''
Put all red color values into csv for good images
'''
path = os.path.join('images', 'good')
df_good_red_vec = pd.DataFrame(columns = df_red_iscc['hex'].values) 
for img_name in os.listdir(path):
    if img_name.startswith('data'):
        continue
    if img_name.startswith('.'):
        continue
       
    path_img = os.path.join(path, img_name)
    print(path_img)
    dict_red, dict_temp_names = img_to_red_vec(path_img)
    print('done')
    for color_hex in df_red_vec.columns:
        if color_hex not in dict_red.keys():
            dict_red[color_hex] = 0
    df_good_red_vec = df_good_red_vec.append(dict_red,ignore_index=True,sort=False)
    print(df_good_red_vec)
    print(df_good_red_vec.size)

603  
4   0.104359  0.198648  0.044717  0.093683  0.005829  0.000000  
5   0.034683  0.126619  0.094119  0.106285  0.059926  0.105820  
6   0.004189  0.027836  0.023654  0.272389  0.122614  0.180109  
7   0.132928  0.124818  0.046162  0.103036  0.077055  0.143065  
8   0.003430  0.119495  0.082804  0.110247  0.059906  0.133704  
9   0.005188  0.127439  0.066829  0.092078  0.105285  0.259556  
10  0.014602  0.068660  0.021236  0.186166  0.146231  0.207968  
11  0.000010  0.000010  0.000228  0.201656  0.069819  0.199275  
12  0.000827  0.087186  0.058316  0.140236  0.263477  0.121895  
13  0.000016  0.000073  0.002022  0.110844  0.055898  0.332390  
14  0.000031  0.031807  0.096433  0.136721  0.127371  0.207850  
15  0.000000  0.000361  0.014460  0.283406  0.069314  0.036975  
16  0.117483  0.202170  0.001823  0.009146  0.163806  0.086088  
17  0.004586  0.166110  0.149130  0.068054  0.069011  0.038112  
18  0.000011  0.002978  0.011145  0.072732  0.013058  0.264198  
19  0.047551  0.069

In [15]:
# all 267 colors
df_vec.shape

(6, 267)

In [15]:
df_red_vec.shape

(26, 47)

In [ ]:
df_vec.to_csv(os.path.join(path, 'data_blood.csv'), index=False)

In [14]:
df_red_vec.to_csv(os.path.join(path, 'data_red_only.csv'), index = False)

In [15]:
df_good_red_vec.to_csv(os.path.join(path, 'data_good_red_only.csv'), index = False)